In [1]:
import pandas as pd
import numpy as np

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
window_size = 60
Npi = 100
bandwidth = 0.1
all_prices = []
for ticker in tickers:
    file_path = f"{folder}{ticker}.csv"
    df = pd.read_csv(file_path, parse_dates=['Date'])
    price_col = 'Adjusted Close' if 'Adjusted Close' in df.columns else 'Adj Close'
    df = df[['Date', price_col]].rename(columns={price_col: ticker})
    all_prices.append(df)

data = all_prices[0]
for df in all_prices[1:]:
    data = data.merge(df, on='Date', how='inner')
data = data.set_index('Date')
data = data.loc["2010-06-01":"2020-05-31"]
log_returns = np.log(data / data.shift(1)).dropna()

def create_sequences(log_returns, window):
    sequences = []
    for i in range(len(log_returns) - window):
        seq = log_returns.iloc[i:i+window].values
        sequences.append(seq)
    return np.array(sequences)

data_series = create_sequences(log_returns, window_size)

def quartic_kernel(x, h):
    norm = np.linalg.norm(x) / h
    return ((1 - norm**2)**2) / h if norm < 1 else 0.0

def Fi(t, xti, x, xtip1, ti, tip1):
    dt1 = tip1 - t
    dt2 = tip1 - ti
    term1 = -np.linalg.norm(xtip1 - x)**2 / (2 * dt1)
    term2 = np.linalg.norm(xtip1 - xti)**2 / (2 * dt2)
    return np.exp(term1 + term2)

def estimate_drift(t, x, xi, data_series, ti, tip1, h):
    num = np.zeros_like(x)
    denom = 0.0
    for sample in data_series:
        if tip1 >= sample.shape[0]:
            continue
        xti = sample[ti]
        xtip1 = sample[tip1]
        Fi_val = Fi(t, xti, x, xtip1, ti, tip1)
        weight = Fi_val
        for j in range(ti):
            weight *= quartic_kernel(xi[j] - sample[j], h)
        num += weight * (xtip1 - x)
        denom += weight
    return (1 / (tip1 - t)) * num / (denom + 1e-8)

def sbts_simulation(data_series, Npi=100, h=0.1):
    M, N, d = data_series.shape
    x = [np.zeros(d)]
    for i in range(N - 1):
        y = x[i].copy()
        for k in range(Npi - 1):
            tpi_k = i + k / Npi
            drift = estimate_drift(tpi_k, y, np.array(x), data_series, ti=i, tip1=i+1, h=h)
            eps = np.random.randn(d)
            y = y + (1 / Npi) * drift + (1 / np.sqrt(Npi)) * eps
        x.append(y)
    return np.array(x)

simulated = sbts_simulation(data_series, Npi=Npi, h=bandwidth)
df_sim = pd.DataFrame(simulated, columns=tickers)
print(df_sim)

/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/2594847526.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/2594847526.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/2594847526.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd6

         AAPL      MSFT       GOOG       AMZN      NFLX
0    0.000000  0.000000   0.000000   0.000000  0.000000
1    0.306872 -0.248457   0.087648  -0.147909  0.131563
2   -0.025126 -0.224654   0.029713   0.072433 -0.205081
3   -1.107726 -1.222794  -0.103220  -0.813675  1.059853
4   -0.355623  0.648119   0.094360   0.320746  1.065554
5   -1.574758 -0.238160   0.876936   0.568719  2.107633
6   -3.776358  0.027708   1.677069   1.269492  3.056535
7   -3.151220  1.544293   0.714245  -0.767039  3.886571
8   -4.743304  0.075525   0.090348  -0.449701  3.364247
9   -4.770519  1.106191  -1.630338   0.698673  3.623190
10  -5.158363  0.504974  -1.490552   2.065626  4.077376
11  -5.610975  0.366797  -1.681719   3.298353  3.946888
12  -6.842331  1.126593  -0.005294   3.808415  2.037985
13  -5.474295  1.273752   0.725146   6.419410  2.419446
14  -6.089874  0.831221   0.865901   4.888514  2.573943
15  -6.418295 -0.211742  -0.264192   4.776578  3.134619
16  -7.645026 -1.026151   0.792378   5.745253  2

In [6]:
returns = df_sim.diff().dropna()

mu = returns.mean().values
cov = returns.cov().values
inv_cov = np.linalg.inv(cov)
ones = np.ones(len(mu))

w_sharpe = inv_cov @ mu
w_sharpe /= np.sum(w_sharpe)

portfolios = pd.DataFrame([w_sharpe],
                          columns=df_sim.columns,
                          index=['MaxSharpe'])
print(portfolios)

               AAPL      MSFT      GOOG      AMZN    NFLX
MaxSharpe  0.462338  0.021288  0.323541 -0.104067  0.2969


In [7]:
import os

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
start_date = '2020-06-01'
end_date = '2020-09-01'
initial_investment = 100000
weights = np.array([0.462338, 0.021288, 0.323541, -0.104067, 0.2969])

close_prices = pd.DataFrame()

for ticker in tickers:
    filepath = os.path.join(folder, f"{ticker}.csv")
    df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
    close_prices[ticker] = df['Adjusted Close']

close_prices = close_prices.loc[start_date:end_date]

start_prices = close_prices.iloc[0]
end_prices = close_prices.iloc[-1]

shares_bought = (initial_investment * weights) / start_prices

final_value = np.sum(shares_bought * end_prices)

print(f"Initial investment: ${initial_investment:,.2f}")
print(f"Final portfolio value after 2 months: ${final_value:,.2f}")
print(f"Return: {(final_value / initial_investment - 1) * 100:.2f}%")

/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/318103123.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/318103123.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/318103123.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Dat

Initial investment: $100,000.00
Final portfolio value after 2 months: $150,458.23
Return: 50.46%


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/318103123.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_74751/318103123.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
